In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
modelo=pd.read_csv('data/modelo.csv',low_memory=False)

In [3]:
modelo.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
modelo.drop(['url', 'sku', 'skus','channel','staticpage','condition','storage','color','search_engine','device_type','campaign_source','country','region','search_term','new_vs_returning','city','region','screen_resolution','operating_system_version','browser_version'], axis=1, inplace=True)

In [5]:
training= pd.read_csv('data/training.csv',low_memory=False)

In [6]:
training.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [7]:
len(modelo[modelo['event']=='conversion'])

7091

In [8]:
modelo[modelo['event']=='conversion']['person'].nunique()

4293

In [9]:
len(modelo[modelo['event']=='checkout'])

65315

In [10]:
modelo[modelo['event']=='checkout']['person'].nunique()

32833

# Armado del modelo

In [11]:
modelo.sort_values('person',inplace=True)

In [12]:
import calendar

modelo['timestamp'] = pd.to_datetime(modelo['timestamp'])

modelo['dia']=modelo['timestamp'].dt.dayofyear
modelo['dias_distintos'] = modelo['person'].map(modelo.groupby(['person'])['dia'].nunique())   
modelo.drop('dia',1,inplace=True)

modelo['modelos_distintos_vistos']= modelo['person'].map( modelo.groupby('person')['model'].nunique())
modelo.drop('model',1,inplace=True)


modelo['total_visitas_usuario'] = modelo.groupby('person')['person'].transform('count')


modelo['visitas_ultimo_mes']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month)==5]).size().unstack()[True])
modelo['visitas_ultimo_mes']=modelo['visitas_ultimo_mes'].fillna(0).astype(int)


modelo['visitas_ultima_semana']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True])
modelo['visitas_ultima_semana']=modelo['visitas_ultima_semana'].fillna(0).astype(int)


modelo['total_checkout']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['checkout'])
modelo['total_checkout']=modelo['total_checkout'].fillna(0).astype(int)


modelo['checkouts_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['checkout'])   
modelo['checkouts_ultimo_mes']=modelo['checkouts_ultimo_mes'].fillna(0).astype(int)


modelo['checkouts_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['checkout'])
modelo['checkouts_ultima_semana']=modelo['checkouts_ultima_semana'].fillna(0).astype(int)

"""
modelo['total_compras']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['conversion'])
modelo['total_compras']=modelo['total_compras'].fillna(0).astype(int)


modelo['compras_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['conversion'])
modelo['compras_ultimo_mes']=modelo['compras_ultimo_mes'].fillna(0).astype(int)


modelo['compras_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['conversion'])
modelo['compras_ultima_semana']=modelo['compras_ultima_semana'].fillna(0).astype(int)
"""

modelo['subscripciones_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['lead'])
modelo['subscripciones_ultimo_mes']=modelo['subscripciones_ultimo_mes'].fillna(0).astype(int)

In [13]:
modelo=modelo.drop_duplicates(subset=['person'])
modelo.drop(['timestamp','event'], 1, inplace=True)

In [14]:
#modelo['total_checkout'].sum(), modelo['total_compras'].sum(),modelo['total_visitas_usuario'].sum(), len(modelo[modelo['total_compras']>0])

KeyError: 'total_compras'

In [15]:
modelo=pd.merge(modelo, training, on='person', how='left')

In [16]:
modelo.head()

,person,dias_distintos,modelos_distintos_vistos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,subscripciones_ultimo_mes,label
0,0008ed71,1,3,6,6,0,3,3,0,0,0.0
1,00091926,22,36,448,448,47,2,2,1,0,NaN
2,00091a7a,1,3,10,0,0,0,0,0,0,NaN
3,000ba417,3,26,206,206,138,6,6,3,0,NaN
4,000c79fe,1,1,17,17,17,1,1,1,0,0.0


In [17]:
a_predecir=modelo[modelo['label'].isnull()]
a_predecir.head()

,person,dias_distintos,modelos_distintos_vistos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,subscripciones_ultimo_mes,label
1,00091926,22,36,448,448,47,2,2,1,0,NaN
2,00091a7a,1,3,10,0,0,0,0,0,0,NaN
3,000ba417,3,26,206,206,138,6,6,3,0,NaN
5,000e4d9e,9,37,411,411,112,1,1,0,0,NaN
6,000e619d,4,23,68,68,0,1,1,0,0,NaN


In [18]:
modelo.drop(modelo[modelo['label'].isnull()].index,inplace=True)
modelo['label']=modelo['label'].astype(int)
modelo.head()

,person,dias_distintos,modelos_distintos_vistos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,subscripciones_ultimo_mes,label
0,0008ed71,1,3,6,6,0,3,3,0,0,0
4,000c79fe,1,1,17,17,17,1,1,1,0,0
10,001802e4,1,2,19,19,19,1,1,1,0,0
12,0019e639,14,26,471,290,0,15,4,0,0,0
14,001b0bf9,1,1,7,7,0,2,2,0,0,0


In [19]:
len(modelo[modelo['label']==1])

980

In [20]:
len(modelo[modelo['label']==0]) 

18434

In [21]:
len(modelo[modelo['label']==1])*100/len(modelo['label']),len(modelo[modelo['label']==0])*100/len(modelo['label'])

(5.047903574739879, 94.95209642526012)

In [22]:
modelo[modelo['label']==1].head(15)

,person,dias_distintos,modelos_distintos_vistos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,subscripciones_ultimo_mes,label
31,003476c0,4,2,43,14,0,0,0,0,0,1
38,003d4cac,3,2,8,0,0,1,0,0,0,1
39,003dd965,1,0,5,5,0,0,0,0,0,1
43,0042421c,2,5,22,0,0,0,0,0,0,1
57,006671fd,1,5,18,0,0,0,0,0,0,1
178,0143d796,1,3,24,24,24,1,1,1,0,1
272,01ed2afc,1,1,7,7,0,0,0,0,0,1
358,02875920,4,3,29,22,17,0,0,0,0,1
501,03754d73,6,1,37,9,9,0,0,0,0,1
559,03c99f07,2,3,43,43,43,0,0,0,0,1


## MACHINE LEARNING

In [23]:
##Ahora empieza ML

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics

X = modelo.drop(['person','label'], axis=1)
y = modelo['label']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

In [25]:
len(X_train),len(X_test),len(y_train),len(y_test)

(14560, 4854, 14560, 4854)

In [26]:
X_train.head()

,dias_distintos,modelos_distintos_vistos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,subscripciones_ultimo_mes
2158,2,1,19,16,16,1,1,1,0
26482,1,8,26,26,26,0,0,0,0
1324,2,22,125,125,0,3,3,0,0
312,6,6,87,87,0,5,5,0,0
5644,1,4,14,14,14,1,1,1,0


In [27]:
y_train.head()

2158     0
26482    1
1324     0
312      0
5644     0
Name: label, dtype: int32

In [28]:
X_test.head()

,dias_distintos,modelos_distintos_vistos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,subscripciones_ultimo_mes
23240,1,9,30,30,0,1,1,0,0
2227,2,6,37,37,4,2,2,0,0
36485,2,11,48,33,33,5,5,5,0
14146,1,2,11,11,0,1,1,0,0
20869,1,1,9,9,0,1,1,0,0


In [29]:
y_test.head()

23240    0
2227     0
36485    0
14146    0
20869    0
Name: label, dtype: int32

In [30]:
len(y_test[y_test == 1]),len(y_test[y_test == 0])

(267, 4587)

## KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier
my_classifier= KNeighborsClassifier()
my_classifier.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [32]:
predictions= my_classifier.predict(X_test)
accuracy_score(y_test,predictions)

0.9429336629583849

In [33]:
my_matrix = metrics.confusion_matrix(y_test, predictions)
my_matrix

array([[4572,   15],
       [ 262,    5]], dtype=int64)

## DECISION TREE

In [34]:
from sklearn import tree
my_classifier2= tree.DecisionTreeClassifier()
my_classifier2.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [35]:
predictions2= my_classifier2.predict(X_test)
accuracy_score(y_test,predictions2)

0.9134734239802225

In [36]:
my_matrix2 = metrics.confusion_matrix(y_test, predictions2)
my_matrix2

array([[4397,  190],
       [ 230,   37]], dtype=int64)

## RANDOM FOREST

In [37]:
from sklearn.ensemble import RandomForestClassifier
my_classifier3 = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
my_classifier3.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=16,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [38]:
predictions3=my_classifier3.predict(X_test)
accuracy_score(y_test, predictions3)

0.9449938195302843

In [39]:
my_matrix3 = metrics.confusion_matrix(y_test, predictions3)
my_matrix3

array([[4587,    0],
       [ 267,    0]], dtype=int64)

## Logistic regression

In [40]:
from sklearn.linear_model import LogisticRegression
my_classifier4 = LogisticRegression()
my_classifier4.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [41]:
predictions4 = my_classifier4.predict(X_test)
accuracy_score(y_test, predictions4)

0.9451998351874743

In [42]:
my_matrix4 = metrics.confusion_matrix(y_test, predictions4)
my_matrix4

array([[4585,    2],
       [ 264,    3]], dtype=int64)

## Neuronal Network

In [43]:
from sklearn.neural_network import MLPClassifier
my_classifier5 = MLPClassifier(max_iter=100000)
my_classifier5.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [44]:
predictions5 = my_classifier5.predict(X_test)
accuracy_score(y_test, predictions5)

0.9451998351874743

In [45]:
my_matrix5 = metrics.confusion_matrix(y_test, predictions5)
my_matrix5

array([[4584,    3],
       [ 263,    4]], dtype=int64)

## Naive Bayes

In [46]:
from sklearn.naive_bayes import GaussianNB
my_classifier6 = GaussianNB()
my_classifier6.fit(X_train,y_train)

GaussianNB(priors=None)

In [47]:
predictions6 = my_classifier6.predict(X_test)
accuracy_score(y_test, predictions6)

0.8871034198599094

In [48]:
metrics.confusion_matrix(y_test, predictions6)

array([[4272,  315],
       [ 233,   34]], dtype=int64)

## SVM

In [49]:
from sklearn import svm
my_classifier7 =svm.SVC(class_weight={1:100,0:5})
my_classifier7.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight={1: 100, 0: 5}, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [50]:
predictions7 = my_classifier7.predict(X_test)
accuracy_score(y_test, predictions7)

0.8916357643180882

In [51]:
metrics.confusion_matrix(y_test, predictions7)

array([[4269,  318],
       [ 208,   59]], dtype=int64)

## AdaBoost

In [52]:
from sklearn.ensemble import AdaBoostClassifier
my_classifier8 = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
my_classifier8.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=42)

In [53]:
predictions8 = my_classifier8.predict(X_test)
accuracy_score(y_test, predictions8)

0.9433456942727647

In [54]:
metrics.confusion_matrix(y_test, predictions8)

array([[4572,   15],
       [ 260,    7]], dtype=int64)

## Voting Classifier

In [71]:
from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(solver='saga', multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=1000,random_state=1,n_jobs =-1)
clf3 = GaussianNB(priors=[0.05,0.95])
clf4= MLPClassifier()
clf5= tree.DecisionTreeClassifier()
#clf6= svm.SVC()
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3) , ('dt',clf5), ('svm',clf4)], voting='soft', n_jobs =-1)
eclf1 = eclf1.fit(X, y)
predictions = eclf1.predict(X_test)
accuracy_score(y_test, predictions)


d:\programas\python\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.984548825710754

In [72]:
metrics.confusion_matrix(y_test, predictions)

array([[4582,    5],
       [  70,  197]], dtype=int64)

In [73]:
clf1 = LogisticRegression(solver='sag', multi_class='ovr', random_state=1 , n_jobs=-1)
clf2 = RandomForestClassifier(n_estimators=1000,random_state=1,n_jobs =-1)
#clf3 = GaussianNB(priors=[0.05,0.95])
#clf4= MLPClassifier()
#clf5= tree.DecisionTreeClassifier()
eclf2 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2)], voting='soft', n_jobs =-1)
eclf2 = eclf1.fit(X, y)
predictions2 = eclf2.predict(X_test)
accuracy_score(y_test, predictions2)

d:\programas\python\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9824886691388546

In [74]:
metrics.confusion_matrix(y_test, predictions2)

array([[4586,    1],
       [  84,  183]], dtype=int64)

## ROC AUC SCORE

In [75]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predictions)

0.8683688391472726

In [76]:
roc_auc_score(y_test, predictions2)

0.8425876255073573

## KAGGLE PREDICTIONS

In [77]:
kaggle= pd.read_csv('data/kaggle.csv',low_memory=False)

In [78]:
kaggle=pd.merge(kaggle, a_predecir, on='person', how='inner')

In [79]:
kaggle.head()

,person,dias_distintos,modelos_distintos_vistos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,subscripciones_ultimo_mes,label
0,4886f805,1,1,9,9,0,1,1,0,0,NaN
1,0297fc1e,59,15,567,173,53,7,2,0,0,NaN
2,2d681dd8,2,3,26,26,16,1,1,1,0,NaN
3,cccea85e,13,57,836,836,270,1,1,0,0,NaN
4,4c8a8b93,5,9,257,257,0,2,2,0,0,NaN


In [80]:
np.savetxt("predictions.csv", eclf1.predict_proba(kaggle.drop(['person','label'], axis=1))  , delimiter=",")